In [1]:
%sh
pip install --upgrade pip

Requirement already up-to-date: pip in /databricks/python3/lib/python3.7/site-packages (20.2.1)

In [2]:
%sh pip install plotly
pip install datetime

Requirement already satisfied: plotly in /databricks/python3/lib/python3.7/site-packages (4.9.0)
Requirement already satisfied: six in /databricks/python3/lib/python3.7/site-packages (from plotly) (1.12.0)
Requirement already satisfied: retrying>=1.3.3 in /databricks/python3/lib/python3.7/site-packages (from plotly) (1.3.3)
Requirement already satisfied: datetime in /databricks/python3/lib/python3.7/site-packages (4.3)
Requirement already satisfied: pytz in /databricks/python3/lib/python3.7/site-packages (from datetime) (2018.9)
Requirement already satisfied: zope.interface in /databricks/python3/lib/python3.7/site-packages (from datetime) (5.1.0)
Requirement already satisfied: setuptools in /databricks/python3/lib/python3.7/site-packages (from zope.interface->datetime) (49.3.1)

In [3]:
from pyspark.sql.functions import col, udf, monotonically_increasing_id
from pyspark.sql.types import IntegerType, TimestampType, StringType, BooleanType, StructType, StructField
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline
import plotly.graph_objects as go
import plotly
import datetime

In [4]:
# File location and type
file_location = "/FileStore/tables/train_project-4.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

#Displaying top 5 rows:
display(df.limit(5))

archived,author,author_cakeday,author_flair_background_color,author_flair_css_class,author_flair_richtext,author_flair_text,author_flair_text_color,author_flair_type,brand_safe,can_gild,contest_mode,created_utc,distinguished,domain,edited,gilded,hidden,hide_score,id,is_crosspostable,is_reddit_media_domain,is_self,is_video,link_flair_css_class,link_flair_richtext,link_flair_text,link_flair_text_color,link_flair_type,locked,media,no_follow,num_comments,num_crossposts,over_18,parent_whitelist_status,permalink,post_hint,preview,retrieved_on,rte_mode,score,secure_media,selftext,send_replies,spoiler,stickied,subreddit,subreddit_id,subreddit_name_prefixed,subreddit_type,suggested_sort,thumbnail,thumbnail_height,thumbnail_width,title,url,whitelist_status
TRUE,codepoet,null,null,null,[],null,null,text,true,true,false,1141171234,null,macgeekery.com,false,0,false,false,2icw,true,false,false,false,null,[],null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/2icw/well_that_was_a_bust/,null,null,null,markdown,0,null,null,TRUE,FALSE,FALSE,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Well That Was a Bust,http://www.macgeekery.com/opinion/well_that_was_a_bust,all_ads
TRUE,scylla,null,null,null,[],null,null,text,true,true,false,1141171723,null,msnbc.msn.com,false,0,false,false,2idn,true,false,false,false,null,[],null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/2idn/holocaust_why_david_irving_shouldnt_be_jailed_and/,null,null,null,markdown,0,null,null,TRUE,FALSE,FALSE,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Holocaust: Why David Irving shouldn’t be jailed ( and it's not because he's in any ways right ),http://www.msnbc.msn.com/id/11569497/site/newsweek/,all_ads
TRUE,tilto,null,null,null,[],null,null,text,true,true,false,1141171939,null,iht.com,false,0,false,false,2ie4,true,false,false,false,null,[],null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/2ie4/google_shares_fall_sharply_as_cfo_announces/,null,null,null,markdown,0,null,null,TRUE,FALSE,FALSE,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,Google shares fall sharply as CFO announces growth is slowing,http://www.iht.com/articles/2006/02/28/business/google.php,all_ads
TRUE,Laibcoms,null,null,null,[],null,null,text,true,true,false,1141172196,null,gameshogun.info,false,0,false,false,2iek,true,false,false,false,null,[],null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/2iek/newsvine_launching_tomorrow/,null,null,null,markdown,0,null,null,TRUE,FALSE,FALSE,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,newsvine launching tomorrow!,http://gameshogun.info/index.php/Tech/2006/03/01/newsvine_launching_tomorrow,all_ads
TRUE,FaeLLe,null,null,null,[],null,null,text,true,true,false,1141172277,null,faelle.com,false,0,false,false,2ies,true,false,false,false,null,[],null,dark,text,false,null,true,0,0,false,all_ads,/r/reddit.com/comments/2ies/voodoopc_to_launch_8tb_media_pc/,null,null,null,markdown,0,null,null,TRUE,FALSE,FALSE,reddit.com,t5_6,r/reddit.com,archived,null,default,null,null,VoodooPC to launch 8TB Media PC,http://www.FaeLLe.com/2006/03/voodoopc-plans-8tb-media-pc.html,all_ads


In [5]:
print("Number of sample is in train dataset:", df.count(), 
     "\nNumber of columns/variables:", len(df.columns))

Number of sample is in train dataset: 12544 
Number of columns/variables: 58

Our target variable is - 'score' in the dataframe. Let's have a look at its nature:

In [7]:
#Analyze the target variable
df.select('score').describe().toPandas()

,summary,score
0,count,12524
1,mean,7.5152967489416085
2,stddev,29.11053033813615
3,min,"""""url"""":""""https://i.redditmedia.com/2GE_8-hy9X..."
4,max,99


There few null values and some urls that need to be removed from the dataset

In [9]:
def data_cleaning(df):
  
    df = df.filter(~df.score.contains('""url"":""https:'))
    df = df.filter(df.score.isNotNull())
    print("Number of Sample after dropping Null Score values",df.count())    
    bol_col = ['brand_safe', 'can_gild', 'is_crosspostable', 'no_follow', 'over_18']
    #casting each of these columne into string to change that can be
    for col_name in bol_col:
      df = df.withColumn(col_name, col(col_name).cast("string"))
    
    return df

In [10]:
#dropping rows that have string "url..https" in te score column and null values
df = df.filter(~df.score.contains('""url"":""https:'))
df = df.filter(df.score.isNotNull())
print("Number of Sample after dropping Null Score values",df.count())

Number of Sample after dropping Null Score values 12519

In [11]:
'''assign target to a separate dataframe
cast the target to integer
drop from the original dataframe'''
target = df.select('score')
target = df.withColumn("score", col("score").cast("integer"))


In [12]:
#storing score in Pandas only for visualizations
score = target.toPandas()['score']

In [13]:
data = go.Histogram(x = score, nbinsx = 20)
layout = go.Layout(title = {'text':'Distribution of scores across the dataset', 'x':0.5,'xanchor':'center', 'yanchor':'top'}, xaxis = dict(title = 'Score'),
                  yaxis = dict(title = 'Count'), width = 500)
go.Figure(data, layout = layout)

From the histogram above it is evident that the number of posts with score greater than 100 are very less

100 to 200 => 190 posts 200 to 300 => 46 posts 300 to 400 => 16 posts 400 to 500 => 1 post 500 to 600 => 3 posts

Therefore, it has to be kept in mind to divide the dataset such that we have each kind of score in train and validation set

In [15]:
df.printSchema()

root
-- archived: string (nullable = true)
-- author: string (nullable = true)
-- author_cakeday: string (nullable = true)
-- author_flair_background_color: string (nullable = true)
-- author_flair_css_class: string (nullable = true)
-- author_flair_richtext: string (nullable = true)
-- author_flair_text: string (nullable = true)
-- author_flair_text_color: string (nullable = true)
-- author_flair_type: string (nullable = true)
-- brand_safe: boolean (nullable = true)
-- can_gild: boolean (nullable = true)
-- contest_mode: boolean (nullable = true)
-- created_utc: integer (nullable = true)
-- distinguished: string (nullable = true)
-- domain: string (nullable = true)
-- edited: boolean (nullable = true)
-- gilded: integer (nullable = true)
-- hidden: boolean (nullable = true)
-- hide_score: boolean (nullable = true)
-- id: string (nullable = true)
-- is_crosspostable: boolean (nullable = true)
-- is_reddit_media_domain: boolean (nullable = true)
-- is_self: boolean (nullable = true)
-- is_video: boolean (nullable = true)
-- link_flair_css_class: string (nullable = true)
-- link_flair_richtext: string (nullable = true)
-- link_flair_text: string (nullable = true)
-- link_flair_text_color: string (nullable = true)
-- link_flair_type: string (nullable = true)
-- locked: boolean (nullable = true)
-- media: string (nullable = true)
-- no_follow: boolean (nullable = true)
-- num_comments: integer (nullable = true)
-- num_crossposts: integer (nullable = true)
-- over_18: boolean (nullable = true)
-- parent_whitelist_status: string (nullable = true)
-- permalink: string (nullable = true)
-- post_hint: string (nullable = true)
-- preview: string (nullable = true)
-- retrieved_on: string (nullable = true)
-- rte_mode: string (nullable = true)
-- score: string (nullable = true)
-- secure_media: string (nullable = true)
-- selftext: string (nullable = true)
-- send_replies: string (nullable = true)
-- spoiler: string (nullable = true)
-- stickied: string (nullable = true)
-- subreddit: string (nullable = true)
-- subreddit_id: string (nullable = true)
-- subreddit_name_prefixed: string (nullable = true)
-- subreddit_type: string (nullable = true)
-- suggested_sort: string (nullable = true)
-- thumbnail: string (nullable = true)
-- thumbnail_height: string (nullable = true)
-- thumbnail_width: string (nullable = true)
-- title: string (nullable = true)
-- url: string (nullable = true)
-- whitelist_status: string (nullable = true)

Columns are either string type, boolean or integer type.
Seaprating these columns for better comprehension of dataset

In [17]:
#string columns
str_col = [var.name for var in df.schema.fields if isinstance(var.dataType,StringType)]

#boolean columns
bol_col = [var.name for var in df.schema.fields if isinstance(var.dataType,BooleanType)]

#integer columns
int_col = [var.name for var in df.schema.fields if isinstance(var.dataType, IntegerType)]

print("Check: No column is left out of the 3 lists: ", len(str_col)+len(bol_col)+len(int_col)==len(df.columns))

Check: No column is left out of the 3 lists: True

In [18]:
def distinct_values(col_list, df):
  distinct_values = {}
  for col_name in col_list:
    distinct_values[col_name] = df.select(col_name).distinct().collect()
    
  return distinct_values

In [19]:
#checking if the column sof boolean datatype have both True and False, that it can be used as a feature
bol_dist_val = distinct_values(bol_col, df)
bol_dist_val

Out[544]: {'brand_safe': [Row(brand_safe=True), Row(brand_safe=False)],
 'can_gild': [Row(can_gild=True), Row(can_gild=False)],
 'contest_mode': [Row(contest_mode=False)],
 'edited': [Row(edited=False)],
 'hidden': [Row(hidden=False)],
 'hide_score': [Row(hide_score=False)],
 'is_crosspostable': [Row(is_crosspostable=True), Row(is_crosspostable=False)],
 'is_reddit_media_domain': [Row(is_reddit_media_domain=False)],
 'is_self': [Row(is_self=False)],
 'is_video': [Row(is_video=False)],
 'locked': [Row(locked=False)],
 'no_follow': [Row(no_follow=True), Row(no_follow=False)],
 'over_18': [Row(over_18=True), Row(over_18=False)]}

brand_safe, can_gild, is_crosspostable, no_follow, over_18 for prediction

In [21]:
bol_col = ['brand_safe', 'can_gild', 'is_crosspostable', 'no_follow', 'over_18']
#casting each of these columne into string to change that can be 
for col_name in bol_col:
  df = df.withColumn(col_name, col(col_name).cast("string"))

In [22]:
str_col

Out[546]: ['archived',
 'author',
 'author_cakeday',
 'author_flair_background_color',
 'author_flair_css_class',
 'author_flair_richtext',
 'author_flair_text',
 'author_flair_text_color',
 'author_flair_type',
 'distinguished',
 'domain',
 'id',
 'link_flair_css_class',
 'link_flair_richtext',
 'link_flair_text',
 'link_flair_text_color',
 'link_flair_type',
 'media',
 'parent_whitelist_status',
 'permalink',
 'post_hint',
 'preview',
 'retrieved_on',
 'rte_mode',
 'score',
 'secure_media',
 'selftext',
 'send_replies',
 'spoiler',
 'stickied',
 'subreddit',
 'subreddit_id',
 'subreddit_name_prefixed',
 'subreddit_type',
 'suggested_sort',
 'thumbnail',
 'thumbnail_height',
 'thumbnail_width',
 'title',
 'url',
 'whitelist_status']

In [23]:
str_col = ['subreddit_type']

In [24]:
str_dist_val = distinct_values(str_col,df)
str_dist_val

Out[548]: {'subreddit_type': [Row(subreddit_type='public'),
 Row(subreddit_type='archived'),
 Row(subreddit_type='restricted')]}

In [25]:
cat_col = bol_col + str_col
int_col = ['num_comments']

In [27]:
%run /Users/shimona.narang@mail.utoronto.ca/feature_engineering

In [28]:
input_df = df
data_to_model = features(df, cat_col, int_col)

categorized varibales function begin
text transform done
input columns for VA are: ['cat_features', 'w2v', 'num_comments']

In [29]:

trainingData = data_to_model.toDF(*['features', 'label'])

In [30]:
%run /Users/shimona.narang@mail.utoronto.ca/test_data


In [31]:
testData = testData.toDF(*['features', 'label'])